In [25]:
from pyMSOO.MFEA.model import MaTGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

In [48]:
import numpy as np
import random
from functools import reduce
import time

from pyMSOO.MFEA.model import AbstractModel
from pyMSOO.utils import Crossover, Mutation, Selection, DimensionAwareStrategy
from pyMSOO.utils.EA import *
from pyMSOO.utils.numba_utils import numba_randomchoice

class model(AbstractModel.model):
    def compile(self, 
        IndClass: Type[Individual],
        tasks: List[AbstractTask], 
        crossover: Crossover.SBX_Crossover, 
        mutation: Mutation.PolynomialMutation, 
        selection: Selection.ElitismSelection, 
        dimension_strategy: DimensionAwareStrategy.AbstractDaS = DimensionAwareStrategy.NoDaS(),
        *args, **kwargs):
        super().compile(IndClass, tasks, crossover, mutation,dimension_strategy, selection, *args, **kwargs)
    
    def fit(self, nb_generations, rmp = 0.3, nb_inds_each_task = 100, evaluate_initial_skillFactor = True, *args, **kwargs) -> List[Individual]:
        super().fit(*args, **kwargs)

        # initialize population
        population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks= self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )

        self.R_o = np.random.rand((len(self.tasks))) 
        self.R_s = np.random.rand((len(self.tasks)))
        self.E_o = np.random.rand((len(self.tasks)))
        self.E_s = np.random.rand((len(self.tasks)))

        # self.R_o = np.full((len(self.tasks)), 100) 
        # self.R_s = np.full((len(self.tasks)), 100)
        # self.E_o = np.full((len(self.tasks)), 100)
        # self.E_s = np.full((len(self.tasks)), 100)

        # save history
        self.history_cost.append([ind.fcost for ind in population.get_solves()])
        
        self.render_process(0, ['Cost'], [self.history_cost[-1]], use_sys= True)

        for epoch in range(nb_generations):
            offsprings = Population(
                self.IndClass,
                nb_inds_tasks = [0] * len(self.tasks), 
                dim = self.dim_uss,
                list_tasks= self.tasks,
            )

            lambda_i = (self.R_o/(self.E_o + 1e-6))/(self.R_s/(self.E_s + 1e-6) + self.R_o/(self.E_o + 1e-6) + 1e-6)
            idx_other = np.where(np.random.rand(len(self.tasks)) < lambda_i)[0]
            idx_same = list(set(range(len(self.tasks))) - set(idx_other))

            self.E_o[idx_other] += nb_inds_each_task
            self.E_s[idx_same] += nb_inds_each_task
            
            # start = time.time()

            for i in idx_other:
                # self.E_o[i] += nb_inds_each_task
                offs = population.__getRandomInds__(nb_inds_each_task)
                # res = np.fromiter(map(self.tasks[i].__call__, offs), float)
                # self.R_o[i] += sum(res < population[i].__getBestIndividual__.fcost)
                current_best = population[i].__getBestIndividual__.fcost
                for o in offs:
                    fcost = self.tasks[i](o)
                    self.R_o[i] += fcost < current_best
                    j = o.skill_factor
                    off = self.dimension_strategy(o, j, population[i].__getRandomItems__())
                    # off.fcost = fcost
                    off.skill_factor = i
                    # print(off.fcost)
                    offsprings.__addIndividual__(off)
            # print(len(idx_same))
            for i in idx_same:
                current_best = population[i].__getBestIndividual__.fcost
                # self.E_s[i] += nb_inds_each_task
                offs = []
                # print("A")
                while len(offs) < nb_inds_each_task:
                    # choose parent 
                    pa, pb = population[i].__getRandomItems__(2)
                    # intra / inter crossover
                    oa, ob = self.crossover(pa, pb, i, i)
                    # mutate
                    oa = self.mutation(oa, return_newInd= True)
                    oa.skill_factor = pa.skill_factor

                    ob = self.mutation(ob, return_newInd= True)    
                    ob.skill_factor = pb.skill_factor
                    
                    offsprings.__addIndividual__(oa)
                    offsprings.__addIndividual__(ob)

                    # self.R_s[i] += self.tasks[i](oa) > current_best
                    # self.R_s[i] += self.tasks[i](ob) > current_best

                    # cnt += 1
                    offs.append(oa)
                    offs.append(ob)

                res = np.fromiter(map(self.tasks[i].__call__, offs), float)
                self.R_s[i] += sum(res < population[i].__getBestIndividual__.fcost)      
            
            # end = time.time()
            # print("B: ", end - start)

            # merge and update rank
            population = population + offsprings
            population.update_rank()

            # selection
            self.selection(population, [nb_inds_each_task] * len(self.tasks))

            # update operators
            self.crossover.update(population = population)
            self.mutation.update(population = population)
            self.dimension_strategy.update(population = population)

            # save history
            self.history_cost.append([ind.fcost for ind in population.get_solves()])

            #print
            self.render_process((epoch+1)/nb_generations, ['Cost'], [self.history_cost[-1]], use_sys= True)
        
        print('\nEND!')

        #solve 
        self.last_pop = population
        return self.last_pop.get_solves() 


In [49]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(10)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()
# tasks, IndClass = WCCI22_benchmark.get_50tasks_benchmark(10)

In [50]:
from pyMSOO.MFEA.model import EBSGA

baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta = 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.5, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 00m 0.60s    0 % [>                   ]  Cost: 1.06E+05  3.09E+05  2.67E+05  3.78E+01  2.96E+09  2.13E+01  7.94E+01  1.64E+04  6.89E+01  5.54E+04  ,  4
6
7
5
4
5
6
5
4
4
Seed: None -- Time: 00m 1.70s    1 % [>                   ]  Cost: 2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.13E+01  2.12E+01  2.12E+01  2.12E+01  ,  5
7
6
6
4
7
6
4
8
6
Seed: None -- Time: 00m 2.90s    2 % [>                   ]  Cost: 2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  ,  4
4
6
5
3
8
6
6
5
4
Seed: None -- Time: 00m 3.87s    3 % [>                   ]  Cost: 2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  ,  4
6
5
7
6
6
7
7
4
6
Seed: None -- Time: 00m 4.85s    4 % [>                   ]  Cost: 2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  2.12E+01  ,  4
7
6
5
7
6
5
4
4
7
Seed: None -- Time: 00m 5.78s    5 % [=>          

KeyboardInterrupt: 

In [41]:
print(baseModel.E_o)

[11100.39040685  4600.62898746 11400.46137089 13200.66177338
  7300.51310563  9300.70456834  8500.22181701  8100.78044644
  6900.95756257  8800.43106713]


In [42]:
print(baseModel.E_s)

[ 6200.21265926 12700.20462918  5900.39822262  4100.70623797
 10000.18128347  8000.92999782  8800.53769328  9200.46881843
 10400.08589775  8500.82239733]


In [43]:
print(baseModel.R_o)

[ 0.96732102  0.14895321  0.98959925  0.944277    0.28815579 23.0481886
  0.64350185  0.64668007  0.68790221  0.90860889]


In [44]:
print(baseModel.R_s)

[ 0.2797658   1.43927715  0.22065739  0.07270277  0.53153977 19.74697752
  0.62658584  0.89064098  1.70077248  0.71703869]
